## Αρχικά θα φορτώσουμε τα κείμενα

In [1]:
def take_files(path):  
    try:
        with open(path, 'r', encoding='utf-8') as file:
            print("File content successfully loaded.")
            return file.read()
    except FileNotFoundError:
        print("Error: The file  was not found. Please make sure the file exists and the path is correct.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [2]:
path1="../texts/text1113.txt"
path2="../texts/text1114.txt"
text1=take_files(path1)
text2=take_files(path2)

File content successfully loaded.
File content successfully loaded.


In [5]:
text1.replace('\n','')

'Κοινό πράγμα. — Αν η κυριότητα του [MASK]  ανήκει σε περισσότερους [MASK] αδιαιρέτου κατ΄ιδανικά [MASK], εφαρμόζονται οι διατάξεις για την κοινωνία.'

In [6]:
text2.replace('\n','')

'Πραγματική δουλεία σε [MASK] η υπέρ του κοινού ακινήτου. — Στο κοινό [MASK] μπορεί να συσταθεί πραγματική δουλεία υπέρ του [MASK] κύριου άλλου ακινήτου και αν ακόμη αυτός είναι [MASK] του ακινήτου που βαρύνεται με τη δουλεία. Το ίδιο ισχύει και για την [MASK] δουλεία πάνω σε ακίνητο υπέρ των εκάστοτε κυρίων κοινού ακινήτου, αν [MASK] από αυτούς είναι κύριος του [MASK] που βαρύνεται με τη δουλεία. '

In [21]:
from transformers import BertTokenizerFast, BertForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
from datasets import Dataset
import torch

# 1. Φόρτωση tokenizer & μοντέλου (GreekBERT από SciFY ή άλλο Multilingual BERT)
model_name = "nlpaueb/bert-base-greek-uncased-v1"
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

# 2. Φόρτωση του custom dataset από txt αρχείο
dataset=[]
try:
    with open("data.txt", 'r', encoding='utf-8') as file:
            print("File content successfully loaded.")
            lines = file.readlines()
            data = [{"text": line.strip()} for line in lines if line.strip()]
except FileNotFoundError:
        print("Error: The file  was not found. Please make sure the file exists and the path is correct.")
except Exception as e:
        print(f"An unexpected error occurred: {e}")
        
dataset = Dataset.from_list(data)

File content successfully loaded.


In [22]:

        


# 3. Tokenization
def tokenize_function(examples):
    return tokenizer(examples["text"], return_special_tokens_mask=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 4. Ομαδοποίηση σε block size (π.χ. 128 tokens)
block_size = 128

def group_texts(examples):
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated["input_ids"])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated.items()
    }
    return result

lm_dataset = tokenized_dataset.map(group_texts, batched=True)

# 5. Data Collator για MLM
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# 6. Training Args
training_args = TrainingArguments(
    output_dir="./greekbert-mlm",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    evaluation_strategy="no"
)

# 7. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# 8. Εκπαίδευση
trainer.train()

# 9. Αποθήκευση μοντέλου
trainer.save_model("./greekbert-mlm")
tokenizer.save_pretrained("./greekbert-mlm")


Map:   0%|          | 0/27541 [00:00<?, ? examples/s]

Map:   0%|          | 0/27541 [00:00<?, ? examples/s]


TypeError: can only concatenate list (not "str") to list